In [61]:
import csv
from charity import Charity

tsv_path = 'data/wikipedia.tsv'

with open(tsv_path, 'r') as tsv_file:
    reader = csv.DictReader(tsv_file, dialect='excel-tab')
    charities = [
        Charity(**row)
        for row in reader
    ]

In [62]:
documents = [charity.description for charity in charities]

In [ ]:
import re

from nltk.tokenize import sent_tokenize
from bert import get_embeddings

embeddings_index = []
embeddings_list = []

for doc_index, doc in enumerate(documents):
    print(doc_index)
    sentences = [
        re.sub('\[\d+\]', '', sent).strip().lower()
        for sent in sent_tokenize(doc)
    ]
    sentence_embeddings = get_embeddings(sentences)
    
    embeddings_index += ([doc_index] * len(sentence_embeddings))
    embeddings_list.append(sentence_embeddings)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158


In [ ]:
import torch

embeddings = torch.cat(embeddings_list)

In [ ]:
import random

import numpy as np

def sample_indices(index, n_positive, n_negative, n_examples):
    all_query_examples = []
    all_positive_examples = []
    all_negative_examples = []
    
    unique_elements = np.unique(index)
    index_indices = torch.arange(len(index))
    
    for _ in range(n_examples):
        elem = random.choice(unique_elements)
        
        elem_indices = index_indices[index == elem]
        query_index = random.choice(elem_indices)
        
        same_document_indices = index_indices[(index == elem)]
        different_document_indices = index_indices[index != elem]
        
        positive_examples = same_document_indices[random.sample(range(0, len(same_document_indices)), n_positive)]
        negative_examples = different_document_indices[random.sample(range(0, len(different_document_indices)), n_negative)]

        all_query_examples.append(query_index)
        all_positive_examples.append(positive_examples)
        all_negative_examples.append(negative_examples)

    query_indices = torch.tensor(all_query_examples)
    positive_example_indices = torch.stack(all_positive_examples)
    negative_example_indices = torch.stack(all_negative_examples)
    
    return query_indices, positive_example_indices, negative_example_indices

In [ ]:
import torch.nn as nn

# model
input_size = 768
output_size = 10
linear_model = nn.Linear(input_size, output_size)

torch.nn.init.xavier_uniform_(linear_model.weight)
linear_model.bias.data.fill_(0.1)
    
loss_function = torch.nn.MarginRankingLoss(margin=1)
optimizer = torch.optim.Adam(linear_model.parameters(), lr=0.1)
cos = nn.CosineSimilarity(dim=2, eps=1e6)

In [ ]:
shuffle_now = False

if shuffle_now:
    shuffled_indices = np.arange(len(embeddings_index))
    np.random.shuffle(shuffled_indices)
    shuffled_indices

In [ ]:
def compute_loss(embeddings, embeddings_index, n_positive, n_negative, n_examples):
    query_indices, positive_example_indices, negative_example_indices = sample_indices(
        embeddings_index,
        n_positive=n_positive,
        n_negative=n_negative,
        n_examples=n_examples,
    )
    
    model_embeddings = linear_model(embeddings)
    embedded_query = model_embeddings[query_indices]
    embedded_positive = model_embeddings[positive_example_indices]
    embedded_negative = model_embeddings[negative_example_indices]
    
    positive_similarity = cos(embedded_query.view(-1, 1, output_size), embedded_positive)    
    negative_similarity = cos(embedded_query.view(-1, 1, output_size), embedded_negative)

    positive_flat = positive_similarity.view(-1, 1).repeat(1, n_negative).view(-1)
    negative_flat = negative_similarity.repeat(1, n_positive).view(-1)
    
    all_ones = torch.ones(len(positive_flat))

    return loss_function(positive_flat, negative_flat, all_ones)

In [ ]:
n_examples = 5000
n_positive = 1
n_negative = 10

train_split = 0.6
split_index = int(len(shuffled_indices) * train_split)
train_indices = shuffled_indices[:split_index]
validation_indices = shuffled_indices[split_index:]

train_embeddings_index = torch.tensor(embeddings_index)[train_indices]
validation_embeddings_index = torch.tensor(embeddings_index)[validation_indices]

for epoch in range(1000):
    train_loss = compute_loss(embeddings, train_embeddings_index, n_positive, n_negative, n_examples)
    
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        validation_loss = compute_loss(
            embeddings,
            validation_embeddings_index,
            n_positive,
            n_negative,
            n_examples,
        )
    
    print('epoch: {} train loss: {} validation loss: {}'.format(
        epoch,
        train_loss,
        validation_loss,
    ))

In [ ]:
from torch.nn.functional import cosine_similarity

search = "dogs"

cos_out = nn.CosineSimilarity(dim=1, eps=1e6)

embedded = get_embeddings([search])
processed = linear_model(embedded)
processed_embeddings = linear_model(embeddings)

similarities = cosine_similarity(
    processed.view(-1, output_size),
    processed_embeddings,
    dim=1)

top_n = similarities.argsort().tolist()[::-1][:5]
doc_indices = torch.tensor(embeddings_index)[top_n]
[charities[i].name for i in doc_indices]

In [12]:
embedded.shape

torch.Size([1, 768])

In [13]:
embeddings.shape

torch.Size([336, 768])

In [17]:
processed_embeddings.shape

torch.Size([336, 10])